In [ ]:
import sys
import math
import numpy as np

def surroundings(h, map_grid, value_matrix, marked, n, unknown_value):
    search_list = []
    for x, y, value in h:
        for x_new, y_new in [[x+1, y], [x, y+1], [x-1, y], [x, y-1]]:
            x_new = int(x_new % len(map_grid))
            y_new = int(y_new % len(map_grid[0]))
            if (x_new, y_new) in marked:
                continue
            if map_grid[x_new][y_new] > -1:
                value_new = value + map_grid[x_new][y_new]
                search_list.append([x_new, y_new, value_new])
                value_matrix[x_new][y_new] = value_new
            elif map_grid[x_new][y_new] == -1:
                value_new = value + unknown_value
                search_list.append([x_new, y_new, value_new])
                value_matrix[x_new][y_new] = value_new
            marked.add((x_new, y_new))
    return search_list 

class Pac(object):
    def __init__(self, x, y, pac_id, type_id, speed_turns_left, ability_cooldown):
        self.x = x
        self.y = y
        self.pac_id = pac_id
        self.type_id = type_id
        self.speed_turns_left = speed_turns_left
        self.ability_cooldown = ability_cooldown
        self.dest = None
    def __eq__(self, another):
        return self.pac_id == another.pac_id
    def __hash__(self):
        return self.pac_id
    def update(self, x, y, type_id, speed_turns_left, ability_cooldown):
        self.type_id = type_id
        self.speed_turns_left = speed_turns_left
        self.ability_cooldown = ability_cooldown
        self.x = x
        self.y = y
    def set_value_matrix(self, map_grid, unknown_value):
        h = [[self.x, self.y, 0]]
        marked = set([(self.x, self.y)])
        self.value_matrix = np.zeros((len(map_grid), len(map_grid[0])))
        n = 1
        while True:
            h = surroundings(h, map_grid, self.value_matrix, marked, n, unknown_value)
            n += 1
            if len(h) == 0 or (n >= 4 and np.max([x[2] for x in h]) > 0):
                break
    def set_opponents(self, oppenent_pacs):
        for pac in oppenent_pacs.values():
            self.value_matrix[(pac.x+1)%len(self.value_matrix)][pac.y] -= 2
            self.value_matrix[(pac.x-1)%len(self.value_matrix)][pac.y] -= 2
            self.value_matrix[pac.x][(pac.y+1)%len(self.value_matrix[0])] -= 2
            self.value_matrix[pac.x][(pac.y-1)%len(self.value_matrix[0])] -= 2
    def set_dest(self, map_grid, oppenent_pacs = {}, unknown_value=1):
        self.set_value_matrix(map_grid, unknown_value)
        self.set_opponents(oppenent_pacs)
#         if int(self.pac_id) == 1:
#             print(self.value_matrix, file=sys.stderr, flush=True)
        self.dest = np.unravel_index(np.argmax(self.value_matrix), map_grid.shape)
    def check_ability(self):
        if self.ability_cooldown == 0:
            if True:
                return "SPEED {}".format(self.pac_id)
        else:
            return None
    def action(self):
        ability = self.check_ability()
        if ability:
            return ability
#         if int(self.pac_id) == 1:
#             print("{} pac {} {} is moving to {} {}".format(self.pac_id, self.x, self.y, self.dest[0], self.dest[1]), file=sys.stderr, flush=True)
        return "MOVE {} {} {}".format(self.pac_id, self.dest[0], self.dest[1])
    
class Grid(object):
    def __init__(self, map_grid):
        # -2 wall, -1 unknown, >=0 pallet value
        self.width = len(map_grid)
        self.height = len(map_grid[0])
        self.map_grid = np.zeros((self.width, self.height))
        for i in range(self.width):
            for j in range(self.height):
                if map_grid[i][j] == "#":
                    self.map_grid[i][j] = -2
                else:
                    self.map_grid[i][j] = -1
#         print(self.map_grid, file=sys.stderr, flush=True)
        self.my_pacs = {}
        self.oppenent_pacs = {}
        
        self.unknown_estimation = 1
        self.step = 0
        
    def update_pacs(self, input_pacs, pacs):
        for x, y, pac_id, type_id, speed_turns_left, ability_cooldown in input_pacs:
            if pac_id not in pacs:
                pacs[pac_id] = Pac(x, y, pac_id, type_id, speed_turns_left, ability_cooldown)
            else:
                pacs[pac_id].update(x, y, type_id, speed_turns_left, ability_cooldown)
    def update_pallets(self, pallets_list):
        my_vision = self.pacs_vision(self.my_pacs)
        self.map_grid[self.map_grid > 0] = -1
        for i in my_vision:
            self.map_grid[i] = 0
        for x, y, value in pallets_list:
            self.map_grid[x][y] = value 
    def update_grid(self, my_pacs, oppenent_pacs, pallets_list):
        self.map_grid[self.map_grid == -3] = 0
        self.update_pacs(my_pacs, self.my_pacs)
        self.update_pacs(oppenent_pacs, self.oppenent_pacs)
        self.update_pallets(pallets_list)
        for pac in self.my_pacs.values():
            self.map_grid[pac.x][pac.y] = -3
        for pac in self.oppenent_pacs.values():
            self.map_grid[pac.x][pac.y] = -3
        self.step += 1
    def update(self, my_pacs, oppenent_pacs, pallets_list):
        self.update_grid(my_pacs, oppenent_pacs, pallets_list)
        self.update_dests(self.my_pacs, self.oppenent_pacs) 
    def set_unknown_estimation(self):
        n = np.sum(grid.map_grid == -1)
        self.unknown_estimation = max(0.1, (n - self.step * len(self.my_pacs.keys())) / n)   
    def update_dests(self, pacs, oppenent_pacs):
        self.set_unknown_estimation()
        for pac in pacs.values():
            pac.set_dest(self.map_grid, oppenent_pacs, self.unknown_estimation)      
    def pacs_vision(self, pacs):
        visible_points = set()
        for pac in pacs.values():
            visible_points |= self.get_visible_points(pac)
        return visible_points
    def get_visible_points(self, pac):
        visible_points = set()
        directions = np.array([[-1,0], [1,0], [0,1], [0,-1]])
        for direction in directions:
            pos = (pac.x, pac.y)
            l = max(self.height, self.width)
            while self.map_grid[pos] != -2 and l > 0:
                visible_points.add(pos)
                pos = tuple((pos + direction) % np.array([self.width, self.height]))
                l -= 1
        return visible_points
    def __repr__(self):
        return self.map_grid.__repr__()
    def action(self):
        return "|".join([p.action() for p in self.my_pacs.values()])

# width: size of the grid
# height: top left corner is (x=0, y=0)
width, height = [int(i) for i in input().split()]
map_grid = []
for i in range(height):
    row = input()  # one line of the grid: space " " is floor, pound "#" is wall
    map_grid.append([x for x in row])
map_grid = np.array(map_grid).transpose()
grid = Grid(map_grid)

pacs = None
# game loop
while True:
    my_score, opponent_score = [int(i) for i in input().split()]
    visible_pac_count = int(input())  # all your pacs and enemy pacs in sight
    my_pacs = []
    oppenent_pacs = []
    for i in range(visible_pac_count):
        inputs = input().split()
        pac_id = int(inputs[0])  # pac number (unique within a team)
        mine = inputs[1] != "0"  # true if this pac is yours
        x = int(inputs[2])  # position in the grid
        y = int(inputs[3])  # position in the grid
        type_id = inputs[4]  # unused in wood leagues
        speed_turns_left = int(inputs[5])  # unused in wood leagues
        ability_cooldown = int(inputs[6])  # unused in wood leagues
        if mine:
            my_pacs.append([x, y, pac_id, type_id, speed_turns_left, ability_cooldown])
        else:
            oppenent_pacs.append([x, y, pac_id, type_id, speed_turns_left, ability_cooldown])
    
    visible_pellet_count = int(input())  # all pellets in sight

    pallets_list = []
    for i in range(visible_pellet_count):
        # value: amount of points this pellet is worth
        pallets_list.append([int(j) for j in input().split()])

    grid.update(my_pacs, oppenent_pacs, pallets_list)

    print(grid.action())
